In [ ]:
'''Train a simple deep NN on the MNIST dataset.
Get to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
nb_epoch = 1

from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def plot_mnist_digit(image, figsize=None):
    """ Plot a single MNIST image."""
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    if figsize:
        ax.set_figsize(*figsize)
    ax.matshow(image, cmap = matplotlib.cm.binary)
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))
    plt.show()

In [ ]:
i = np.random.randint(len(X_train))
plot_mnist_digit(X_train[i])
print(i, ':', y_train[i])

In [ ]:
def plot_10_by_10_images(images, figsize=None):
    """ Plot 100 MNIST images in a 10 by 10 table. Note that we crop
    the images so that they appear reasonably close together.  The
    image is post-processed to give the appearance of being continued."""
    fig = plt.figure(figsize=figsize)
    #images = [image[3:25, 3:25] for image in images]
    #image = np.concatenate(images, axis=1)
    for x in range(10):
        for y in range(10):
            ax = fig.add_subplot(10, 10, 10*y+x+1)
            ax.matshow(images[10*y+x], cmap = matplotlib.cm.binary)
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))
    plt.show()
    
plot_10_by_10_images(X_train, figsize=(10,10))

In [ ]:
def to_features(X):
    return X.reshape(-1, 784).astype("float32") / 255.0

def to_image(X):
    return (X*255.0).astype('uint8').reshape(-1, 28, 28)

print((X_train[0]-(to_image(to_features(X_train[0])))).max())

In [ ]:
print(X_train[0].shape)
print(to_features(X_train[0]).shape)
print(to_image(to_features(X_train))[0].shape)

In [ ]:
# the data, shuffled and split between train and test sets
X_train = to_features(X_train)
X_test = to_features(X_test)
print(X_train.shape, 'train samples')
print(X_test.shape, 'test samples')

In [ ]:
# The labels need to be transformed into class indicators
from keras.utils import np_utils
nb_classes = 10
y_train_cat = np_utils.to_categorical(y_train, nb_classes=nb_classes)
y_test_cat = np_utils.to_categorical(y_test, nb_classes=nb_classes)
print(y_train_cat.shape, 'train labels')
print(y_test_cat.shape, 'test labels')

## Simple Multi-Layer Perceptron

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
mlp = Sequential()
mlp.add(Dense(output_dim=512, input_dim=784, init='uniform'))
mlp.add(Activation('sigmoid'))
mlp.add(Dense(output_dim=nb_classes, input_dim=512, init='uniform'))
mlp.add(Activation('softmax'))

In [ ]:
mlp.compile(loss='categorical_crossentropy', optimizer='SGD')

In [ ]:
mlp.fit(X_train, y_train_cat, batch_size=128, nb_epoch=nb_epoch, verbose=1, show_accuracy=True)

In [ ]:
mlp.evaluate(X_test, y_test_cat, show_accuracy=True)

In [ ]:
j = np.random.randint(len(X_test))
plot_mnist_digit(X_test[j].reshape(28,28))
prediction = mlp.predict_classes(X_test[j:j+1], verbose=False)[0]
print(j, ':', '\tpredict:', prediction, '\tactual:', y_test[j])

## A Deeper MLP

In [ ]:
mlp2 = Sequential()
mlp2.add(Dense(output_dim=512, input_dim=784, init='uniform'))
mlp2.add(Activation('sigmoid'))
mlp2.add(Dense(output_dim=512, input_dim=512, init='uniform'))
mlp2.add(Activation('sigmoid'))
mlp2.add(Dense(output_dim=nb_classes, input_dim=512, init='uniform'))
mlp2.add(Activation('softmax'))

In [ ]:
mlp2.compile(loss='categorical_crossentropy', optimizer='SGD')

In [ ]:
mlp2.fit(X_train, y_train_cat, batch_size=128, nb_epoch=nb_epoch, show_accuracy=True, verbose=1)

In [ ]:
mlp2.evaluate(X_test, y_test_cat, show_accuracy=True)

## A More Modern MLP

In [ ]:
from keras.layers.core import Dropout
mlp3 = Sequential()
mlp3.add(Dense(output_dim=512, input_dim=784, init='uniform'))
mlp3.add(Activation('relu'))
mlp3.add(Dropout(0.2))
mlp3.add(Dense(output_dim=512, input_dim=512, init='uniform'))
mlp3.add(Activation('relu'))
mlp3.add(Dropout(0.2))

mlp3.add(Dense(output_dim=512, input_dim=512, init='uniform'))
mlp3.add(Activation('relu'))
mlp3.add(Dropout(0.2))

mlp3.add(Dense(output_dim=nb_classes, input_dim=512, init='uniform'))
mlp3.add(Activation('softmax'))

In [ ]:
mlp3.compile(loss='categorical_crossentropy', optimizer='SGD')

In [ ]:
mlp3.fit(X_train, y_train_cat, batch_size=128, nb_epoch=nb_epoch, show_accuracy=True, verbose=1)

In [ ]:
mlp3.evaluate(X_test, y_test_cat, show_accuracy=True)

In [ ]:
#open('mlp2_model.yaml', 'w').write(mlp3.to_yaml())
#mlp3.save_weights('mlp3_weights.hdf5')

## Simple Autoencoder

In [ ]:
#creating the autoencoder
from keras.layers import containers
from keras.layers.core import AutoEncoder
ae = Sequential()
encoder = containers.Sequential()
encoder.add(Dense(input_dim=784, output_dim=512, init='uniform'))
encoder.add(Activation('relu'))
encoder.add(Dropout(0.2))
decoder = containers.Sequential()
decoder.add(Dense(512, 784, init='uniform'))
decoder.add(Activation('relu'))
ae.add(AutoEncoder(encoder=encoder, decoder=decoder,
                   output_reconstruction=True))

In [ ]:
ae.compile(loss='mean_squared_error', optimizer='SGD')

In [ ]:
ae.fit(X_train, X_train, batch_size=128, nb_epoch=np_epoch, show_accuracy=True, verbose=1)

In [ ]:
# plot example reconstruction here